# Job Description Analysis

**Author**: Jerome Wynne (jeromewynne@das-ltd.co.uk)

**Created**: 5th November 2020  

**Environment**: wmchack

**Summary**: Classifying job roles: An NLP challenge using NHS Job descriptions, matching to Professional groups to (i.e. using FEDIP & AphA definition of analytical or civil service GORS/GSS/GDS).

---

### Write list of JSON files to Feather format 

In [101]:
import pandas as pd
from glob import glob
import json
from time import time
import html
import nltk

In [14]:
def load_job_descriptions():
    ''' Reads all .json files in /data into dataframe.

        The dataframe constructed can be viewed in /logs/info.log.

        Returns:
            pd.DataFrame: dataframe where each row corresponds to one 
                            file.
    '''
    fps = glob(r'..\data\*.json')
    fps = [fp for fp in fps if 'example' not in fp] # remove example.json
    list_of_page_dct = []
    for fp in fps: # read all of the files into a list of dicts
        with open(fp, 'r', encoding='utf-8') as f:
            list_of_page_dct += [json.load(f)]
    
    # flatten the list of dicts to a DataFrame
    df = pd.json_normalize(list_of_page_dct)

    return(df)

In [26]:
df = load_job_descriptions()
df.to_feather('../data/20201105_' + str(int(time())) + '_nhs_job_descriptions.feather')

---

## Load source data from Feather file, display essential information

In [27]:
fp = glob('../data/*.feather')[0]
df = pd.read_feather(fp)

In [34]:
# display head

df.head(3)

,@context,@type,datePosted,description,employmentType,industry,title,url,validThrough,baseSalary.@type,...,hiringOrganization.url,jobLocation.@type,jobLocation.address.@type,jobLocation.address.addressCountry,jobLocation.address.addressLocality,jobLocation.address.postalCode,jobLocation.geo.@type,jobLocation.geo.latitude,jobLocation.geo.longitude,hiringOrganization.logo
0,http://schema.org,JobPosting,2019-08-09T15:53:44+0100,&lt;p&gt;Stockton Hall is a highly specialist ...,FULL_TIME,Healthcare,Senior Staff Nurse £3000 Welcome Bonus,https://www.jobs.nhs.uk/xi/vacancy/915703981,2020-11-27T23:59:59+0000,MonetaryAmount,...,https://www.jobs.nhs.uk/xi/agency_info/?agency...,Place,PostalAddress,GB,"Stockton on the Forest, York",YO32 9UN,GeoCoordinates,53.9954,-1.00281,None
1,http://schema.org,JobPosting,2019-08-21T07:54:40+0100,&lt;p&gt;Birmingham Community Healthcare NHS F...,TEMPORARY,Healthcare,Bank Level 8a Advanced Clinical Practitioner,https://www.jobs.nhs.uk/xi/vacancy/915718669,2021-09-20T23:59:59+0100,MonetaryAmount,...,https://www.jobs.nhs.uk/xi/agency_info/?agency...,Place,PostalAddress,GB,Birmingham,B7 4BN,GeoCoordinates,52.4873,-1.88581,https://www.jobs.nhs.uk/xi/db_image/logo/12565...
2,http://schema.org,JobPosting,2019-08-21T13:41:57+0100,&lt;p&gt;&lt;strong&gt;Are you looking for a c...,TEMPORARY,Healthcare,Band 5 Bank Staff Nurse RN RGN,https://www.jobs.nhs.uk/xi/vacancy/915719754,2021-05-04T23:59:59+0100,MonetaryAmount,...,https://www.jobs.nhs.uk/xi/agency_info/?agency...,Place,PostalAddress,GB,Leicester/Leicestershire/Rutland,LE16 7BN,GeoCoordinates,52.4872,-.931093,None


In [32]:
# get fields, shape, and memory usage

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4706 entries, 0 to 4705
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   @context                             4706 non-null   object
 1   @type                                4706 non-null   object
 2   datePosted                           4706 non-null   object
 3   description                          4706 non-null   object
 4   employmentType                       4706 non-null   object
 5   industry                             4706 non-null   object
 6   title                                4706 non-null   object
 7   url                                  4706 non-null   object
 8   validThrough                         4706 non-null   object
 9   baseSalary.@type                     4706 non-null   object
 10  baseSalary.currency                  4706 non-null   object
 11  baseSalary.value.@type               4706 n

---

## Identify 'analyst' jobs for later reference

In [160]:
df[['title', 'url']][df['title'].str.contains('Analyst')]

,title,url
255,Transformation and Integration Manager - Progr...,https://www.jobs.nhs.uk/xi/vacancy/916192104
964,Band 7 EPR Configuration Analyst,https://www.jobs.nhs.uk/xi/vacancy/916230379
1277,Epidemiology and Information Analyst,https://www.jobs.nhs.uk/xi/vacancy/916235893
1415,Business Analyst,https://www.jobs.nhs.uk/xi/vacancy/916236874
1574,Senior Information Analyst,https://www.jobs.nhs.uk/xi/vacancy/916238178
1726,Skids Data Analyst - Project Manager,https://www.jobs.nhs.uk/xi/vacancy/916239321
1923,Workforce Planning Support Analyst,https://www.jobs.nhs.uk/xi/vacancy/916240652
1945,Principal Analyst,https://www.jobs.nhs.uk/xi/vacancy/916240832
2080,Information Analyst,https://www.jobs.nhs.uk/xi/vacancy/916241821
2104,Analyst Programmer,https://www.jobs.nhs.uk/xi/vacancy/916241979


---

## Retain only description, view some examples

In [46]:
ser_desc = df['description'] 
ser_desc.index = df['url']

In [42]:
ser_desc[0]

'&lt;p&gt;Stockton Hall is a highly specialist hospital where patients requiring a medium secure care setting can maximise value from therapeutic interventions. Our services have produced excellent recovery results for patients and they are able to progress to low secure or locked rehabilitation if possible.&lt;/p&gt;\n&lt;p&gt;The facility is split across seven wards as follows:&lt;/p&gt;\n&lt;p&gt;16 bedded female ward - Mental Health and Personality Disorder&lt;br /&gt; 16 bedded male ward - Personality Disorder&lt;br /&gt; 8 bedded elderly male ward - for those with a settled in presentation in need of rehabilitation for mental illness and/or personality disorders&lt;br /&gt; Two 24 bedded male wards for mental health focus&lt;/p&gt;\n&lt;p&gt;The role:&lt;/p&gt;\n&lt;p&gt;Here at Stockton Hall Hospital in York, we have a vacancy for Staff Nurse – Grade 4.&lt;/p&gt;\n&lt;p&gt;Job Purpose:&lt;/p&gt;\n&lt;ul&gt;\n&lt;li&gt;To provide high profile leadership on a shift basis&lt;/li&gt

In [41]:
ser_desc[100]

'&lt;p&gt;Are you a Registered Nurse looking for flexibility, a good rate of pay and the opportunity to make a difference to people’s lives?&lt;/p&gt;\n&lt;p&gt;If you are an enthusiastic, caring and compassionate individual looking to work flexibly in a supportive environment then we’d love to hear from you!&lt;/p&gt;\n&lt;p&gt;&lt;strong&gt;EPUT are recruiting Nurses to work on our Trust bank across Essex covering a large geographical region.&lt;/strong&gt;&lt;/p&gt;\n&lt;p&gt;&lt;strong&gt;At EPUT, we put our people’s talents at the heart of improving lives – come and join us, together we can change lives. &lt;/strong&gt;&lt;/p&gt;\n&lt;p&gt;&lt;strong&gt;We are looking for Nurses who will embrace of values of being compassionate, empowering and open. If this sounds like you, apply now. &lt;/strong&gt;&lt;/p&gt;\n&lt;p&gt;Furthermore, we are seeking individuals that are able to work flexibly across a variety of shift patterns to suit your work life balance.&lt;/p&gt;\n&lt;p&gt;In re

In [51]:
ser_desc.iloc[100:101]

url
https://www.jobs.nhs.uk/xi/vacancy/916134096    &lt;p&gt;Are you a Registered Nurse looking fo...
Name: description, dtype: object

---

## Define normalization sequence

We can see that there are HTML entities scattered throughout the text.

Preprocessing pipeline:
1. Strip HTML entities and tags.
2. Strip newline characters.
3. Cast to lowercase.
4. Stem. 
5. Stop word removal. 

In [148]:
def preprocess(ser_desc: pd.Series):
    ser_desc = ser_desc.apply(lambda value: html.unescape(value)) # replace entities
    ser_desc = ser_desc.str.replace(pat='<\/?[^>]*>', repl='') # strip html tags
    ser_desc = ser_desc.str.replace(pat='\\n', repl=' ') # strip newline chars
    ser_desc = ser_desc.str.lower()
    ser_desc = ser_desc.apply(lambda value: nltk.word_tokenize(value)) # splits string into list of tokens
    ser_desc = ser_desc.apply(lambda value: nltk.Text(value))
    #porter = nltk.PorterStemmer()
    #ser_desc = ser_desc.apply(lambda lst: [porter.stem(t) for t in lst]) 
    df_desc = ser_desc.to_frame()
    df_desc.columns = ['text']
    return df_desc

In [158]:
def get_keyword_counts(df_text: pd.DataFrame):
    ''' Creates a data frame containing counts of keywords.
    ''' 
    keywords = sorted(['analyst', 'analysis', 'data', 'prediction', 'predictive', 'sql', 'python', 'r',
                       'digital', 'software', 'automation', 'automate'])
    df_out = df_text.apply(axis=1, 
                           func=lambda rec: [rec['text'].count(kw) for kw in keywords],
                           result_type='expand')
    df_out.columns = keywords
    return df_out

In [149]:
df_text = ser_desc.pipe(preprocess)

In [155]:
df_count = get_keyword_counts(df_text)

In [157]:
df_count.sum(axis=0)

analysis      150
analyst        52
data          557
prediction      0
predictive      1
python          0
r              45
sql            31
dtype: int64